In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline

In [3]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [79]:
data = pd.read_csv("/home/recsys/dataset/train_csv/2020-05-06-00/train/part-00000-608bc593-d570-4042-a4f7-08772a7ce0be-c000.csv")

In [81]:
data.head()

,0d41f79cba32a5ec4362596540226fea1ee6b374,1588695721000,0,15146432668795494,zh,cn,dfzx,36.189.230.152,all,list,...,0.3,0.4,4,48,"12,11,6",0.38731022674426124,0.07618922551472425,健康.1,1.0,0.07610270888098117
0,6203b617aaed707de770076ce1c5ab6ed88899c2,1588695734000,0,15143735143232983,zh,cn,dfzx,220.166.45.39,all,list,...,0,0,0,48,"20,17,12,3,1",0.366875,0.141336,"抗日解放战争,日军,历史","1.0,1.0,1.0","0.20690942408904975,0.22276387136986134,0.1413..."
1,33efd103694b5d64649669d690ac1a085888294a,1588697531000,0,15151121951526806,zh,cn,dfzx,183.186.34.62,all,list,...,0,0,0,48,18,0.955740,0.001677,"垃圾分类六大问题,国内",1.0,0.0
2,7f006c17affd6411a978201bd5c5a59399ddc96a,1588696575000,1,15132194118108735,zh,cn,dfzx,124.23.133.109,all,list,...,0,0,0,48,3,0.084070,0.070709,"春晚,娱乐","1.0,1.0","0.08244826635650455,0.07075970818415672"
3,7f006c17affd6411a978201bd5c5a59399ddc96a,1588696714000,0,15132194118108735,zh,cn,dfzx,124.23.133.109,all,list,...,0,0,0,48,3,0.072256,0.041787,"春晚,娱乐","1.0,1.0","0.03871084765726776,0.04184489514221828"
4,9f272590f01634c9c46b33d354c8150aff315287,1588695745000,0,15154982779248679,zh,cn,dfzx,171.39.1.153,all,list,...,0,0,4,48,13,0.860488,NaN,"日本,娱乐","1.0,1.0",0.0


In [5]:
data.dtypes

request_id               object
report_time              object
action                   object
uid                      object
t_language               object
t_country                object
t_pid                    object
t_ip                     object
t_channel                object
t_scene                  object
t_ctype                  object
t_action                 object
t_l_country              object
t_l_province             object
t_l_city                 object
u_umi                    object
u_uli                    object
u_usi                    object
u_uvi                    object
u_level                  object
ct_count                 object
page_pt                  object
data_source              object
item_id                  object
cp_doc_type              object
cp_l1_category           object
cp_interests             object
cp_location              object
cp_publisher             object
cp_publish_time          object
cp_list_image_count      object
cp_image

In [6]:
choose_columns = ['report_time', 'uid', 'action', 't_channel', 't_scene',
                  't_action', 't_l_country', 't_l_province', 't_l_city', 'u_uli', 'u_umi',
                  'u_level', 'item_id', 'cp_l1_category', 'cp_interests', 'cp_location', 'cp_publisher',
                  'cp_publish_time', 'cp_media_level', 'cp_life_hour', 'rs_channel', 'rs_gactr', 'rs_taginfo'
                 ]

In [7]:
choose_data = data.loc[data['t_scene'] == 'list'][choose_columns]

In [8]:
choose_data.reset_index(drop=True, inplace=True)

In [9]:
bin(int(133152.0))

'0b100000100000100000'

In [10]:
choose_data.head()

,report_time,uid,action,t_channel,t_scene,t_action,t_l_country,t_l_province,t_l_city,u_uli,...,cp_l1_category,cp_interests,cp_location,cp_publisher,cp_publish_time,cp_media_level,cp_life_hour,rs_channel,rs_gactr,rs_taginfo
0,1588285267000,15204077775533240,6,all,list,1,中国,湖北省,咸宁市,:蒋鼎文:0.0016957074;:刘国梁:0.7737707;:陈静:0.0330095...,...,体育:1.0,球迷:1.0;运动员:1.0,NaN,羽毛球爱好者,1.58826e+12,0,36,4096,NaN,1:运动员:1.0:0.0;1:球迷:1.0:0.0;2:体育:1.0:
1,1588285268000,15204077775533240,6,all,list,1,中国,湖北省,咸宁市,:蒋鼎文:0.0016957074;:刘国梁:0.7737707;:陈静:0.0330095...,...,健康:1.0,红酒:1.0;火龙果:1.0;猕猴桃:1.0;山楂:1.0;饮食:1.0,NaN,健康一言堂,1.58814e+12,0,92,524288,0.0718714,2:健康:1.0:0.071748359555482
2,1588285269000,15204077775533240,6,all,list,1,中国,湖北省,咸宁市,:蒋鼎文:0.0016957074;:刘国梁:0.7737707;:陈静:0.0330095...,...,健康:1.0,心脑血管:1.0;鸡蛋:1.0;高血压:1.0;胆固醇:1.0;血压:1.0;高胆固醇:1.0,NaN,一起来侃健康,1.5882e+12,0,384,4,0.0940531,1:高血压:1.0:0.1347411723899103;1:心脑血管:1.0:0.1204...
3,1588285271000,15204077775533240,6,all,list,1,中国,湖北省,咸宁市,:蒋鼎文:0.0016957074;:刘国梁:0.7737707;:陈静:0.0330095...,...,健康:1.0,心脑血管:1.0;高血压:1.0;香蕉:1.0,NaN,北青网,1.58804e+12,3,72,4,0.023838,1:高血压:1.0:0.04373069350774798;1:心脑血管:1.0:0.039...
4,1588285271000,15204077775533240,6,all,list,1,中国,湖北省,咸宁市,:蒋鼎文:0.0016957074;:刘国梁:0.7737707;:陈静:0.0330095...,...,历史:1.0,戴笠:1.0;日本:1.0;间谍:1.0;二战:1.0;妓女:1.0;抗日解放战争:1.0,重庆市:0.3;重庆市_重庆市:0.7,第一书记,1.58807e+12,5,180,33,0.0844841,1:抗日解放战争:1.0:0.1784256148550514;2:历史:1.0:0.084...


In [12]:
choose_data['action'] = choose_data['action'].astype('int').fillna(6).apply(lambda x: x - 6)

In [13]:
def parser_interest(item):
    out = ''
    item = str(item)
    if item != 'nan':
        for part in item.split(';'):
            out += part.split(':')[1]+"|"
        out = out[:-1]
    return out

In [14]:
def parser_category(item):
    out = ''
    item = str(item)
    if item != 'nan':
        out += item.split(';')[0].split(':')[0]
    return out

In [15]:
def parser_cp_interest(item):
    out = ''
    item = str(item)
    if item != 'nan':
        for part in item.split(';'):
            out += part.split(':')[0]+"|"
        out = out[:-1]
    return out

In [16]:
def parser_location(item):
    out = ''
    item = str(item)
    if item != 'nan':
        for part in item.split(';'):
            out += part.split(':')[0]+"|"
        out = out[:-1]
    return out

In [17]:
def parser_rs_tag_info(item):
    out = ''
    item = str(item)
    if item != '':
        try:
            for part in item.split(';'):
                if len(part.split(':')) == 1:
                    out += part+"|"
                else:
                    out += part.split(':')[1]+"|"
            if len(out) > 1:
                return out[:-1]
        except:
            print(item)
    return out

In [18]:
def parser_rs_channel(item):
    out = ''
    if not item or str(item) == 'NaN':
        return out
    item = bin(int(item))
    size = len(item)
    for i in range(2, len(item)):
        if item[i] == '1':
            out += str(size -i)+"|"
    return out

In [19]:
print(parser_rs_channel(133152.0))

18|12|6|


In [20]:
choose_data['rs_channel'] = choose_data['rs_channel'].fillna(0).apply(parser_rs_channel)

In [21]:
choose_data['u_uli'] = choose_data['u_uli'].apply(parser_interest)
choose_data['u_umi'] = choose_data['u_umi'].apply(parser_interest)
# choose_data['u_usi'] = choose_data['u_usi'].apply(parser_interest)
# choose_data['u_uvi'] = choose_data['u_uvi'].apply(parser_interest)

In [22]:
choose_data['u_level'] = choose_data['u_level'].fillna(0).astype('int')
choose_data['cp_life_hour'] = choose_data['cp_life_hour'].fillna(0).astype('int')
choose_data['rs_gactr'] = choose_data['rs_gactr'].fillna(0).astype('float')
choose_data['t_action'] = choose_data['t_action'].fillna(0).astype('int')
# choose_data['cp_list_image_count'] = choose_data['cp_list_image_count'].fillna(0).astype('int')

In [23]:
choose_data['uid'] = choose_data['uid'].astype('str')
choose_data['t_action'] = choose_data['t_action'].astype('str')
choose_data['cp_media_level'] = choose_data['cp_media_level'].astype('str')
choose_data['cp_interests'] = choose_data['cp_interests'].astype('str')
choose_data['rs_taginfo'] = choose_data['rs_taginfo'].astype('str')

In [24]:
choose_data['cp_l1_category'] = choose_data['cp_l1_category'].apply(parser_category)

In [25]:
choose_data['cp_interests'] = choose_data['cp_interests'].apply(parser_cp_interest)

In [26]:
choose_data['cp_location'] = choose_data['cp_location'].apply(parser_location)

In [27]:
choose_data['rs_taginfo'] = choose_data['rs_taginfo'].fillna("").apply(parser_rs_tag_info)

In [28]:
choose_data['report_time'] = pd.to_datetime(choose_data['report_time'].astype('int'), utc=True, unit = 'ms').dt.tz_convert('Asia/Shanghai')
choose_data['cp_publish_time'] = pd.to_datetime(choose_data['cp_publish_time'].astype('int'), utc=True, unit = 'ms').dt.tz_convert('Asia/Shanghai')

In [29]:
choose_data['cp_publish_time']

0         2020-05-01 00:32:00+08:00
1         2020-04-29 13:21:00+08:00
2         2020-04-30 07:46:00+08:00
3         2020-04-28 11:10:00+08:00
4         2020-04-28 17:13:00+08:00
                     ...           
5305143   2020-05-03 12:44:00+08:00
5305144   2020-05-02 01:45:00+08:00
5305145   2020-05-02 13:18:00+08:00
5305146   2020-05-03 09:20:00+08:00
5305147   2020-05-02 01:45:00+08:00
Name: cp_publish_time, Length: 5305148, dtype: datetime64[ns, Asia/Shanghai]

In [30]:
choose_data.dtypes

report_time        datetime64[ns, Asia/Shanghai]
uid                                       object
action                                     int64
t_channel                                 object
t_scene                                   object
t_action                                  object
t_l_country                               object
t_l_province                              object
t_l_city                                  object
u_uli                                     object
u_umi                                     object
u_level                                    int64
item_id                                   object
cp_l1_category                            object
cp_interests                              object
cp_location                               object
cp_publisher                              object
cp_publish_time    datetime64[ns, Asia/Shanghai]
cp_media_level                            object
cp_life_hour                               int64
rs_channel          

In [31]:
choose_data.head(5)

,report_time,uid,action,t_channel,t_scene,t_action,t_l_country,t_l_province,t_l_city,u_uli,...,cp_l1_category,cp_interests,cp_location,cp_publisher,cp_publish_time,cp_media_level,cp_life_hour,rs_channel,rs_gactr,rs_taginfo
0,2020-05-01 06:21:07+08:00,15204077775533240,0,all,list,1,中国,湖北省,咸宁市,蒋鼎文|刘国梁|陈静|银行|侯耀华|医保|黄埔军校|奥运会|井冈山|癌症肿瘤|军衔|陈赓|孔...,...,体育,球迷|运动员,,羽毛球爱好者,2020-05-01 00:32:00+08:00,0.0,36,13|,0.000000,运动员|球迷|体育
1,2020-05-01 06:21:08+08:00,15204077775533240,0,all,list,1,中国,湖北省,咸宁市,蒋鼎文|刘国梁|陈静|银行|侯耀华|医保|黄埔军校|奥运会|井冈山|癌症肿瘤|军衔|陈赓|孔...,...,健康,红酒|火龙果|猕猴桃|山楂|饮食,,健康一言堂,2020-04-29 13:21:00+08:00,0.0,92,20|,0.071871,健康
2,2020-05-01 06:21:09+08:00,15204077775533240,0,all,list,1,中国,湖北省,咸宁市,蒋鼎文|刘国梁|陈静|银行|侯耀华|医保|黄埔军校|奥运会|井冈山|癌症肿瘤|军衔|陈赓|孔...,...,健康,心脑血管|鸡蛋|高血压|胆固醇|血压|高胆固醇,,一起来侃健康,2020-04-30 07:46:00+08:00,0.0,384,3|,0.094053,高血压|心脑血管|血压|健康
3,2020-05-01 06:21:11+08:00,15204077775533240,0,all,list,1,中国,湖北省,咸宁市,蒋鼎文|刘国梁|陈静|银行|侯耀华|医保|黄埔军校|奥运会|井冈山|癌症肿瘤|军衔|陈赓|孔...,...,健康,心脑血管|高血压|香蕉,,北青网,2020-04-28 11:10:00+08:00,3.0,72,3|,0.023838,高血压|心脑血管|健康
4,2020-05-01 06:21:11+08:00,15204077775533240,0,all,list,1,中国,湖北省,咸宁市,蒋鼎文|刘国梁|陈静|银行|侯耀华|医保|黄埔军校|奥运会|井冈山|癌症肿瘤|军衔|陈赓|孔...,...,历史,戴笠|日本|间谍|二战|妓女|抗日解放战争,重庆市|重庆市_重庆市,第一书记,2020-04-28 17:13:00+08:00,5.0,180,6|1|,0.084484,抗日解放战争|历史


In [32]:
choose_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5305148 entries, 0 to 5305147
Data columns (total 23 columns):
 #   Column           Dtype                        
---  ------           -----                        
 0   report_time      datetime64[ns, Asia/Shanghai]
 1   uid              object                       
 2   action           int64                        
 3   t_channel        object                       
 4   t_scene          object                       
 5   t_action         object                       
 6   t_l_country      object                       
 7   t_l_province     object                       
 8   t_l_city         object                       
 9   u_uli            object                       
 10  u_umi            object                       
 11  u_level          int64                        
 12  item_id          object                       
 13  cp_l1_category   object                       
 14  cp_interests     object                       
 15

In [33]:
choose_data.columns.values

array(['report_time', 'uid', 'action', 't_channel', 't_scene', 't_action',
       't_l_country', 't_l_province', 't_l_city', 'u_uli', 'u_umi',
       'u_level', 'item_id', 'cp_l1_category', 'cp_interests',
       'cp_location', 'cp_publisher', 'cp_publish_time', 'cp_media_level',
       'cp_life_hour', 'rs_channel', 'rs_gactr', 'rs_taginfo'],
      dtype=object)

In [34]:
sparse_features = ['uid', 't_channel', 't_scene', 't_action',
       't_l_country', 't_l_province', 't_l_city', 
        'item_id', 'cp_interests', 'cp_publisher', 'cp_media_level']

In [41]:
dense_features = ['u_level', 'cp_life_hour', 'rs_gactr']

In [36]:
choose_data.dtypes

report_time        datetime64[ns, Asia/Shanghai]
uid                                       object
action                                     int64
t_channel                                 object
t_scene                                   object
t_action                                  object
t_l_country                               object
t_l_province                              object
t_l_city                                  object
u_uli                                     object
u_umi                                     object
u_level                                    int64
item_id                                   object
cp_l1_category                            object
cp_interests                              object
cp_location                               object
cp_publisher                              object
cp_publish_time    datetime64[ns, Asia/Shanghai]
cp_media_level                            object
cp_life_hour                               int64
rs_channel          

In [37]:
target = ['action']

In [38]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [39]:
for feat in sparse_features:
    lbe = LabelEncoder()
    print(feat)
    choose_data[feat] = lbe.fit_transform(choose_data[feat])

uid
t_channel
t_scene
t_action
t_l_country
t_l_province
t_l_city
item_id
cp_interests
cp_publisher
cp_media_level


In [42]:
mms = MinMaxScaler(feature_range=(0, 1))
choose_data[dense_features] = mms.fit_transform(choose_data[dense_features])

In [45]:
choose_data.tail(300)

,report_time,uid,action,t_channel,t_scene,t_action,t_l_country,t_l_province,t_l_city,u_uli,...,cp_l1_category,cp_interests,cp_location,cp_publisher,cp_publish_time,cp_media_level,cp_life_hour,rs_channel,rs_gactr,rs_taginfo
5304848,2020-05-03 18:51:54+08:00,162742,0,0,0,0,2,111,167,,...,娱乐,18329,,912,2020-05-03 09:29:00+08:00,9,0.000000,7|,0.645832,娱乐
5304849,2020-05-03 18:51:56+08:00,162742,0,0,0,0,2,111,167,,...,健康,4221,,2732,2020-05-03 08:21:00+08:00,9,0.034483,6|,0.646300,健康
5304850,2020-05-03 18:52:12+08:00,162742,0,0,0,0,2,111,167,,...,社会,4747,江苏省|江苏省_南京市|江苏省_南京市_秦淮区,2699,2020-05-03 14:43:00+08:00,5,0.034483,23|4|,0.641667,社会
5304851,2020-05-03 18:52:13+08:00,162742,0,0,0,0,2,111,167,,...,搞笑,11906,,6577,2020-05-03 11:36:00+08:00,3,1.000000,6|,0.645926,搞笑
5304852,2020-05-03 18:52:18+08:00,162742,0,0,0,0,2,111,167,,...,动物,11322,,9394,2020-05-01 22:13:00+08:00,9,0.034483,6|,0.646204,动物
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5305143,2020-05-03 18:33:50+08:00,162756,0,0,0,0,2,84,242,,...,国际,8166,,9591,2020-05-03 12:44:00+08:00,5,0.034483,6|3|,0.645889,美国|美国对外贸易|国际
5305144,2020-05-03 18:33:51+08:00,162756,0,0,0,0,2,84,242,,...,国际,7973,,12546,2020-05-02 01:45:00+08:00,1,0.087644,3|,0.649423,美国|国际
5305145,2020-05-03 18:33:51+08:00,162756,0,0,0,0,2,84,242,,...,国际,8022,台湾省,9591,2020-05-02 13:18:00+08:00,5,0.034483,20|6|3|,0.646484,美国|国际
5305146,2020-05-03 18:33:52+08:00,162756,0,0,0,0,2,84,242,,...,健康,32435,广东省|广东省_广州市,7204,2020-05-03 09:20:00+08:00,1,0.068966,23|4|,0.642239,健康


In [46]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [47]:
def gen_pad_seq(index):
    key2index = {}
    def split(x):
        key_ans = x.split('|')
        for key in key_ans:
            if key not in key2index:
                key2index[key] = len(key2index) + 1
        return list(map(lambda x: key2index[x], key_ans))
    index_list = list(map(split, choose_data[index].values))
    index_length = np.array(list(map(len, index_list)))
    max_len = max(index_length)
    index_list = pad_sequences(index_list, maxlen=max_len, padding='post', )
    return key2index, index_list, max_len

In [48]:
uli_key2index, uli_index_list, uli_max_len  = gen_pad_seq('u_uli')
umi_key2index, umi_index_list, umi_max_len = gen_pad_seq('u_umi')

MemoryError: Unable to allocate 10.7 GiB for an array with shape (5305148, 540) and data type int32

In [ ]:
cp_location_key2index, cp_location_index_list, cp_location_max_len = gen_pad_seq('cp_location')
rs_taginfo_key2index, rs_taginfo_index_list, rs_taginfo_index__max_len = gen_pad_seq('rs_taginfo')

In [ ]:
rs_channel_key2index, rs_channel_index_list, rs_channel_max_len = gen_pad_seq('rs_channel')

In [ ]:
choose_data.info()

In [ ]:
from deepctr.inputs import SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names
from deepctr.models import DeepFM

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, choose_data[feat].nunique(), embedding_dim=8)
                              for feat in sparse_features] + [DenseFeat(feat, 1, ) for feat in dense_features]

In [ ]:
fixlen_feature_columns

In [ ]:
varlen_feature_columns = [VarLenSparseFeat(SparseFeat('u_uli', vocabulary_size=len(uli_key2index) + 1, embedding_dim=128), maxlen=uli_max_len, combiner='mean',
                                                   weight_name=None), VarLenSparseFeat(SparseFeat('rs_taginfo', vocabulary_size=len(rs_taginfo_key2index) + 1, embedding_dim=128), maxlen=rs_taginfo_index__max_len, combiner='mean',
                                                   weight_name=None), VarLenSparseFeat(SparseFeat('rs_channel', vocabulary_size=len(rs_channel_key2index) + 1, embedding_dim=8), maxlen=rs_channel_max_len, combiner='mean',
                                                   weight_name=None)]

In [ ]:
varlen_feature_columns

In [ ]:
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
feature_names

In [ ]:
model_input = {name: choose_data[name] for name in feature_names}
model_input['u_uli'] = uli_index_list
model_input['rs_taginfo'] = rs_taginfo_index_list
model_input['rs_channel'] = rs_channel_index_list

In [ ]:
choose_data.dtypes

In [ ]:
feature_names

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )
history = model.fit(model_input, choose_data[target].values,
                    batch_size=256, epochs=5, verbose=2, validation_split=0.2, )

In [ ]:
pred_ans = model.predict(model_input[-1000:], batch_size=256)

In [ ]:
model_input.keys()

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(choose_data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [ ]:
min(test.index.values.tolist())

In [ ]:
min(train.index.values.tolist())

In [ ]:
uli_item_map = dict()
for i in range(len(uli_index_list)):
    uli_item_map[i] = uli_index_list[i]

In [ ]:
len(uli_item_map)

In [ ]:
# model_input = {name: choose_data[name] for name in sparse_features}  #
# model_input["uli"] = uli_index_list
# model_input["genres_weight"] = np.random.randn(data.shape[0], max_len, 1)

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')

In [ ]:
model.compile("adam", "binary_crossentropy",
                  metrics=['binary_crossentropy'], )

In [ ]:
history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=2, verbose=2, validation_split=0.2, )


In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [8]:
import os

In [9]:
path = "/home/recsys/dataset/train/2020-05-01-00"

In [10]:
def load_dataset(dir_path):
    files = os.listdir(dir_path)
    print(dir_path)
    dataset = tf.data.TFRecordDataset(filenames = [dir_path + '/' + file for file in files])
    return dataset

In [11]:
train = load_dataset(path+"/train")

/home/recsys/dataset/train/2020-05-01-00/train


In [75]:
from tensorflow import feature_column

In [72]:
def parse_function(example_proto):
    dics = {'action': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=6),
            'uid': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
            'item_id': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             't_scene': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             't_location': tf.io.VarLenFeature(dtype=tf.string),
#             't_action': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=1),
#             't_channel': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             't_scene': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             'u_level': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             'u_uli': tf.io.VarLenFeature(dtype=tf.string),
#             'u_umi': tf.io.VarLenFeature(dtype=tf.string),
#             'u_usi': tf.io.VarLenFeature(dtype=tf.string),
#             'cp_word_count': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=0),
#             'cp_media_level': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=0),
#             'cp_publisher': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=''),
#             'cp_is_local_publisher': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=0),
#             'cp_location': tf.io.VarLenFeature(dtype=tf.string),
#             'cp_is_local': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=0),
#             'cp_l1_category': tf.io.VarLenFeature(dtype=tf.string),
#             'cp_life_hour': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=72),
#             'cp_interests': tf.io.VarLenFeature(dtype=tf.string),
#             'rs_gactr': tf.io.FixedLenFeature(shape=(), dtype=tf.float32, default_value=0),
#             'rs_channel': tf.io.VarLenFeature(dtype=tf.string),
            'rs_taginfo': tf.io.VarLenFeature(dtype=tf.string),
           }
    # parse all features in a single example according to the dics
    parsed_example = tf.io.parse_single_example(example_proto, dics)
    target = parsed_example['action']
    feature_columns = []
    
    del parsed_example['action']
    return parsed_example, target

In [73]:
new_dataset = train.map(parse_function)

In [77]:
for item, target in new_dataset.take(1):
    print('item:', item['rs_taginfo'])
    print('target', target)

item: SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]
 [3]], shape=(4, 1), dtype=int64), values=tf.Tensor(
[b'\xe7\xbe\x8e\xe5\x9b\xbd'
 b'\xe5\x9b\xbd\xe9\x99\x85\xe6\x97\xb6\xe6\x94\xbf'
 b'\xe4\xbf\x84\xe7\xbd\x97\xe6\x96\xaf' b'\xe5\x9b\xbd\xe9\x99\x85'], shape=(4,), dtype=string), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64))
target tf.Tensor(0, shape=(), dtype=int64)


In [76]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

<MapDataset shapes: ({rs_taginfo: (None,), item_id: (), uid: ()}, ()), types: ({rs_taginfo: tf.string, item_id: tf.string, uid: tf.string}, tf.int64)>